# MERRA2 Monthly Granularity Test
- Download
- Trim
- Create Cutout
- Generate Wind Outputs

## 1. Imports and Create Dataset

In [2]:
import geodata
import xarray as xr
import logging
logging.basicConfig(level=logging.INFO)
DS_monthly = geodata.Dataset(module="merra2",
					 years=slice(2008, 2008),
					 months=slice(1,6),
                     weather_data_config = "surface_flux_monthly")  


INFO:geodata.dataset:Directory /Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2 found, checking for completeness.
INFO:geodata.dataset:Directory complete.


In [3]:
## Verify Dataset
DS_monthly.__dict__

In [3]:
if DS_monthly.prepared == False:
	DS_monthly.get_data()

## 2.Trim

In [3]:
## Variables before trimming
ds = xr.open_dataset(DS_monthly.downloadedFiles[1][1])
ds.data_vars

In [3]:
# No longer need downloadedFiles = True
DS_monthly.trim_variables()

In [3]:
## Variables after trimming
ds = xr.open_dataset(DS_monthly.downloadedFiles[1][1])
list(ds.data_vars)

In [3]:
# True if variables in trimmed dataset equal variables in config 
DS_monthly.weatherconfig['variables'] == list(ds.data_vars)

# 3. Create Cutout

In [7]:
cutout = geodata.Cutout(name="tokyo-2008-1-test",
                       module="merra2",
                       weather_data_config="surface_flux_monthly",
                       xs=slice(138.5, 139.5),
                       ys=slice(35, 36),
                       years=slice(2008, 2008),
                       months=slice(1,1))

In [7]:
cutout.prepare()

INFO:geodata.cutout:Cutout (tokyo-2008-1-test, /Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts) not found or incomplete.
INFO:geodata.preparation:Starting preparation of cutout 'tokyo-2008-1-test'
INFO:geodata.datasets.merra2:MultiIndex([(2008, 1)],
           names=['year', 'month'])
INFO:geodata.datasets.merra2:[(2008, 1)]
INFO:geodata.preparation:1 tasks have been collected. Starting running them on all processors.
INFO:geodata.datasets.merra2:Opening `/Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2/2008/MERRA2_300.tavgM_2d_flx_Nx.200801.nc4`
INFO:geodata.preparation:Merging variables into monthly compound files
INFO:geodata.preparation:Cutout 'tokyo-2008-1-test' has been successfully prepared


In [9]:
## Verify Cutout Contents
cutout.__dict__

{'name': 'tokyo-2008-1-test',
 'cutout_dir': '/Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts/tokyo-2008-1-test',
 'prepared': True,
 'meta_append': 0,
 'config': 'surface_flux_monthly',
 'dataset_module': <module 'geodata.datasets.merra2' from '/usr/local/lib/python3.7/site-packages/geodata-0.0.2-py3.7.egg/geodata/datasets/merra2.py'>,
 'meta': <xarray.Dataset>
 Dimensions:     (time: 1, x: 2, y: 3, year-month: 1)
 Coordinates:
   * y           (y) float64 35.0 35.5 36.0
   * x           (x) float64 138.8 139.4
   * time        (time) datetime64[ns] 2008-01-01
     lon         (x) float64 138.8 139.4
     lat         (y) float64 35.0 35.5 36.0
   * year-month  (year-month) MultiIndex
   - year        (year-month) int64 2008
   - month       (year-month) int64 1
 Data variables:
     *empty*
 Attributes:
     History:                           Original file generated: Tue Jun 30 21...
     Filename:                          MERRA2_300.tavgM_2d_flx_Nx.200804.nc4
     Com

In [10]:
## Verify Cutout Coords
cutout.coords

# 4. Generate Outputs

In [12]:
ds_wind = geodata.convert.wind(
                 cutout, 
                 turbine='Suzlon_S82_1.5_MW', 
                 smooth=True, 
                 var_height='lml')
ds_wind

<xarray.DataArray (time: 1, y: 3, x: 2)>
array([[[0.00419508, 0.02942151],
        [0.01058039, 0.05267962],
        [0.10489164, 0.0398446 ]]])
Coordinates:
    lat      (y) float64 35.0 35.5 36.0
    lon      (x) float64 138.8 139.4
  * x        (x) float64 138.8 139.4
  * y        (y) float64 35.0 35.5 36.0
  * time     (time) datetime64[ns] 2008-01-01T00:30:00

In [13]:
df_wind = ds_wind.to_dataframe(name='wind')
df_wind

lat      lon      wind
time                y    x                               
2008-01-01 00:30:00 35.0 138.750  35.0  138.750  0.004195
                         139.375  35.0  139.375  0.029422
                    35.5 138.750  35.5  138.750  0.010580
                         139.375  35.5  139.375  0.052680
                    36.0 138.750  36.0  138.750  0.104892
                         139.375  36.0  139.375  0.039845

In [15]:
ds_windwpd = geodata.convert.windwpd(
                cutout, 
                turbine='Vestas_V66_1750kW', 
                var_height='lml')
ds_windwpd

<xarray.DataArray (time: 1, y: 3, x: 2)>
array([[[ 1.88176  , 26.917198 ],
        [ 7.6360803, 49.887188 ],
        [90.27525  , 37.614548 ]]], dtype=float32)
Coordinates:
    lat      (y) float64 35.0 35.5 36.0
    lon      (x) float64 138.8 139.4
  * x        (x) float64 138.8 139.4
  * y        (y) float64 35.0 35.5 36.0
  * time     (time) datetime64[ns] 2008-01-01T00:30:00

In [16]:
df_windwpd = ds_windwpd.to_dataframe(name='windwpd')
df_windwpd

lat      lon    windwpd
time                y    x                                
2008-01-01 00:30:00 35.0 138.750  35.0  138.750   1.881760
                         139.375  35.0  139.375  26.917198
                    35.5 138.750  35.5  138.750   7.636080
                         139.375  35.5  139.375  49.887188
                    36.0 138.750  36.0  138.750  90.275253
                         139.375  36.0  139.375  37.614548

In [18]:
ds_windspd = geodata.convert.windspd(
                cutout, 
                turbine='Vestas_V66_1750kW', 
                var_height='lml')
ds_windspd

<xarray.DataArray 'wnd67m' (time: 1, y: 3, x: 2)>
array([[[1.4499094, 3.5014899],
        [2.3455117, 4.3108544],
        [5.3523173, 3.9228833]]], dtype=float32)
Coordinates:
    lat      (y) float64 35.0 35.5 36.0
    lon      (x) float64 138.8 139.4
  * x        (x) float64 138.8 139.4
  * y        (y) float64 35.0 35.5 36.0
  * time     (time) datetime64[ns] 2008-01-01T00:30:00
Attributes:
    long_name:  extrapolated 67 m wind speed using log ratio, from variable h...
    units:      m s**-1

In [19]:
df_windspd = ds_windwpd.to_dataframe(name='windspd')
df_windspd

lat      lon    windspd
time                y    x                                
2008-01-01 00:30:00 35.0 138.750  35.0  138.750   1.881760
                         139.375  35.0  139.375  26.917198
                    35.5 138.750  35.5  138.750   7.636080
                         139.375  35.5  139.375  49.887188
                    36.0 138.750  36.0  138.750  90.275253
                         139.375  36.0  139.375  37.614548